In [1]:
# autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import lifelines
import pycox
import xgboost as xgb
from sklearn.model_selection import train_test_split

from xgbse import XGBSEKaplanNeighbors, XGBSEKaplanTree, XGBSEDebiasedBCE, XGBSEBootstrapEstimator, XGBSEStackedWeibull
from xgbse.converters import convert_data_to_xgb_format, convert_to_structured, convert_y
from xgbse.non_parametric import get_time_bins

from benchmark import BenchmarkLifelines, BenchmarkXGBoost, BenchmarkXGBSE, BenchmarkPysurvival
#from pysurvival.models.survival_forest import ConditionalSurvivalForestModel

# setting seed
np.random.seed(42)

In [3]:
def split_train_test_valid(dataf, test_size=.2, valid_size=.01, random_state=1):
    df_train, df_test = train_test_split(dataf, test_size=test_size, random_state=random_state)
    df_train, df_valid = train_test_split(df_train, test_size=valid_size, random_state=random_state)
    return df_train, df_valid,  df_test

## Data

In [4]:
from pycox.datasets import metabric
metabric_df = metabric.read_df()
df_train, df_valid, df_test = split_train_test_valid(metabric_df)

T_train = df_train.duration
E_train = df_train.event
TIME_BINS = time_bins = get_time_bins(T_train, E_train, size=30)

Dataset 'metabric' not locally available. Downloading...
Done


## Model

Let us fit a model and check performance.

In [5]:
results = pd.DataFrame()

## Lifelines

In [6]:

bench_lifelines = BenchmarkLifelines(lifelines.CoxPHFitter(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "Cox-PH"
)

In [7]:
bench_lifelines.train()

In [8]:
cox_lifelines_results = bench_lifelines.test()
cox_lifelines_results

{'model': 'Cox-PH',
 'c-index': 0.6216153910637942,
 'ibs': 0.1536047604322741,
 'dcal_pval': 0.5666124375648869,
 'dcal_max_dev': 0.02580576869001855,
 'training_time': 0.24448299407958984,
 'inference_time': 0.003534078598022461}

In [9]:
results = results.append(cox_lifelines_results, ignore_index=True)

In [10]:
bench_lifelines = BenchmarkLifelines(lifelines.WeibullAFTFitter(),
        df_train.assign(duration = np.where(df_train.duration == 0, 1, df_train.duration)),
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "Weibull AFT"
)

In [11]:
bench_lifelines.train()
weibull_aft_results = bench_lifelines.test()
weibull_aft_results

{'model': 'Weibull AFT',
 'c-index': 0.6221812389974013,
 'ibs': 0.15412227274195364,
 'dcal_pval': 0.6667597865912567,
 'dcal_max_dev': 0.02424563941655307,
 'training_time': 0.283588171005249,
 'inference_time': 0.0049610137939453125}

In [12]:
results = results.append(weibull_aft_results, ignore_index=True)

## XGBoost

In [13]:
bench_xgboost = BenchmarkXGBoost(xgb,
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGB - AFT",
        "survival:aft",
)

In [14]:
bench_xgboost.train()
xgboost_aft_results = bench_xgboost.test()
xgboost_aft_results

{'model': 'XGB - AFT',
 'c-index': 0.5995054069913656,
 'ibs': nan,
 'dcal_pval': nan,
 'dcal_max_dev': nan,
 'training_time': 0.04438591003417969,
 'inference_time': 0.0007588863372802734}

In [15]:
results = results.append(xgboost_aft_results, ignore_index=True)

In [16]:
bench_xgboost = BenchmarkXGBoost(xgb,
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGB - Cox",
        "survival:cox",
)
bench_xgboost.train()
xgboost_cox_results = bench_xgboost.test()
xgboost_cox_results

{'model': 'XGB - Cox',
 'c-index': 0.6166275463157013,
 'ibs': nan,
 'dcal_pval': nan,
 'dcal_max_dev': nan,
 'training_time': 0.09560084342956543,
 'inference_time': 0.0006392002105712891}

In [17]:
results = results.append(xgboost_cox_results, ignore_index=True)

## XGB SE

In [18]:
bench_xgboost_embedding = BenchmarkXGBSE(XGBSEDebiasedBCE(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGBSE - Debiased BCE",
        "survival:aft",
)
bench_xgboost_embedding.train()
xgboost_bce_results = bench_xgboost_embedding.test()
xgboost_bce_results

{'model': 'XGBSE - Debiased BCE',
 'c-index': 0.626791851789756,
 'ibs': 0.16544718820181464,
 'dcal_pval': 0.1282159506899636,
 'dcal_max_dev': 0.03267038833783695,
 'training_time': 3.1647579669952393,
 'inference_time': 0.0900719165802002}

In [19]:
results = results.append(xgboost_bce_results, ignore_index=True)

In [20]:
bench_xgboost_embedding = BenchmarkXGBSE(XGBSEKaplanNeighbors(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGBSE - Kaplan Neighbors",
        "survival:aft",
)
bench_xgboost_embedding.train()
xgboost_kn_results = bench_xgboost_embedding.test()
xgboost_kn_results

{'model': 'XGBSE - Kaplan Neighbors',
 'c-index': 0.605205800989186,
 'ibs': 0.1626562183639951,
 'dcal_pval': 0.5884955731415418,
 'dcal_max_dev': 0.022632881520071785,
 'training_time': 0.15366601943969727,
 'inference_time': 0.11072587966918945}

In [21]:
results = results.append(xgboost_kn_results, ignore_index=True)

In [22]:
bench_xgboost_embedding = BenchmarkXGBSE(XGBSEKaplanTree(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGBSE - Kaplan Tree",
        "survival:aft",
)
bench_xgboost_embedding.train()
xgboost_kt_results = bench_xgboost_embedding.test()
xgboost_kt_results

{'model': 'XGBSE - Kaplan Tree',
 'c-index': 0.589791684131109,
 'ibs': 0.16482640216664288,
 'dcal_pval': 0.18030272928223431,
 'dcal_max_dev': 0.03586345646948624,
 'training_time': 0.04992818832397461,
 'inference_time': 0.0022470951080322266}

In [23]:
results = results.append(xgboost_kt_results, ignore_index=True)

In [24]:
base_tree = XGBSEKaplanTree()

bench_xgboost_embedding = BenchmarkXGBSE(
    XGBSEBootstrapEstimator(base_tree, n_estimators=100),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGBSE - Bootstrap Trees",
        "survival:aft",
)
bench_xgboost_embedding.train()
xgboost_bootstrap_results = bench_xgboost_embedding.test()
xgboost_bootstrap_results

{'model': 'XGBSE - Bootstrap Trees',
 'c-index': 0.6242769720848352,
 'ibs': 0.1549380947038349,
 'dcal_pval': 0.5634196757562135,
 'dcal_max_dev': 0.023621561823879617,
 'training_time': 6.165475845336914,
 'inference_time': 0.300656795501709}

In [25]:
results = results.append(xgboost_bootstrap_results, ignore_index=True)

In [26]:
bench_xgboost_embedding = BenchmarkXGBSE(XGBSEStackedWeibull(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGBSE - Stacked Weibull",
        "survival:aft",
)
bench_xgboost_embedding.train()
xgboost_weibull_results = bench_xgboost_embedding.test()
xgboost_weibull_results

{'model': 'XGBSE - Stacked Weibull',
 'c-index': 0.629809707435661,
 'ibs': 0.16175955116529236,
 'dcal_pval': 0.14584322589592966,
 'dcal_max_dev': 0.04453524769696715,
 'training_time': 0.5253489017486572,
 'inference_time': 0.009682178497314453}

In [27]:
results = results.append(xgboost_weibull_results, ignore_index=True)

In [28]:
results.sort_values("c-index", ascending=False).round(3)

,c-index,dcal_max_dev,dcal_pval,ibs,inference_time,model,training_time
8,0.630,0.045,0.146,0.162,0.010,XGBSE - Stacked Weibull,0.525
4,0.627,0.033,0.128,0.165,0.090,XGBSE - Debiased BCE,3.165
7,0.624,0.024,0.563,0.155,0.301,XGBSE - Bootstrap Trees,6.165
1,0.622,0.024,0.667,0.154,0.005,Weibull AFT,0.284
0,0.622,0.026,0.567,0.154,0.004,Cox-PH,0.244
3,0.617,NaN,NaN,NaN,0.001,XGB - Cox,0.096
5,0.605,0.023,0.588,0.163,0.111,XGBSE - Kaplan Neighbors,0.154
2,0.600,NaN,NaN,NaN,0.001,XGB - AFT,0.044
6,0.590,0.036,0.180,0.165,0.002,XGBSE - Kaplan Tree,0.050


In [29]:
results.sort_values("c-index", ascending=False).round(3).set_index('model').to_markdown()

'| model                    |   c-index |   dcal_max_dev |   dcal_pval |     ibs |   inference_time |   training_time |\n|:-------------------------|----------:|---------------:|------------:|--------:|-----------------:|----------------:|\n| XGBSE - Stacked Weibull  |     0.63  |          0.045 |       0.146 |   0.162 |            0.01  |           0.525 |\n| XGBSE - Debiased BCE     |     0.627 |          0.033 |       0.128 |   0.165 |            0.09  |           3.165 |\n| XGBSE - Bootstrap Trees  |     0.624 |          0.024 |       0.563 |   0.155 |            0.301 |           6.165 |\n| Weibull AFT              |     0.622 |          0.024 |       0.667 |   0.154 |            0.005 |           0.284 |\n| Cox-PH                   |     0.622 |          0.026 |       0.567 |   0.154 |            0.004 |           0.244 |\n| XGB - Cox                |     0.617 |        nan     |     nan     | nan     |            0.001 |           0.096 |\n| XGBSE - Kaplan Neighbors |     0.605 